# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [1]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [2]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

In [3]:
san.fs_search(0.012, 2, 20, [Time1, Position1], 1.5, 8, 0.012)

(0.5, 0.5)
(0.5, 1.0)
(0.5, 1.5)
(0.5, 2.0)
(0.5, 2.5)
(0.5, 3.0)
(0.5, 3.5)
(0.5, 4.0)
(0.5, 4.5)
(0.5, 5.0)
(0.5, 5.5)
(0.5, 6.0)
(0.5, 6.5)
(0.5, 7.0)
(0.5, 7.5)
(0.5, 8.0)
(0.5, 8.5)
(0.5, 9.0)
(0.5, 9.5)
(1.0, 0.5)
(1.0, 1.0)
(1.0, 1.5)
(1.0, 2.0)
(1.0, 2.5)
(1.0, 3.0)
(1.0, 3.5)
(1.0, 4.0)
(1.0, 4.5)
(1.0, 5.0)
(1.0, 5.5)
(1.0, 6.0)
(1.0, 6.5)
(1.0, 7.0)
(1.0, 7.5)
(1.0, 8.0)
(1.0, 8.5)
(1.0, 9.0)
(1.0, 9.5)
(1.5, 0.5)
(1.5, 1.0)
(1.5, 1.5)
(1.5, 2.0)
(1.5, 2.5)
(1.5, 3.0)
(1.5, 3.5)
(1.5, 4.0)
(1.5, 4.5)
(1.5, 5.0)
(1.5, 5.5)
(1.5, 6.0)
(1.5, 6.5)
(1.5, 7.0)
(1.5, 7.5)
(1.5, 8.0)
(1.5, 8.5)
(1.5, 9.0)
(1.5, 9.5)
(2.0, 0.5)
(2.0, 1.0)
(2.0, 1.5)
(2.0, 2.0)
(2.0, 2.5)
(2.0, 3.0)
(2.0, 3.5)
(2.0, 4.0)
(2.0, 4.5)
(2.0, 5.0)
(2.0, 5.5)
(2.0, 6.0)
(2.0, 6.5)
(2.0, 7.0)
(2.0, 7.5)
(2.0, 8.0)
(2.0, 8.5)
(2.0, 9.0)
(2.0, 9.5)
(2.5, 0.5)
(2.5, 1.0)
(2.5, 1.5)
(2.5, 2.0)
(2.5, 2.5)
(2.5, 3.0)
(2.5, 3.5)
(2.5, 4.0)
(2.5, 4.5)
(2.5, 5.0)
(2.5, 5.5)
(2.5, 6.0)
(2.5, 6.5)
(2.5, 7.0)
(2.5, 7.5)

(19.0, 5.5)
(19.0, 6.0)
(19.0, 6.5)
(19.0, 7.0)
(19.0, 7.5)
(19.0, 8.0)
(19.0, 8.5)
(19.0, 9.0)
(19.0, 9.5)
(19.5, 0.5)
(19.5, 1.0)
(19.5, 1.5)
(19.5, 2.0)
(19.5, 2.5)
(19.5, 3.0)
(19.5, 3.5)
(19.5, 4.0)
(19.5, 4.5)
(19.5, 5.0)
(19.5, 5.5)
(19.5, 6.0)
(19.5, 6.5)
(19.5, 7.0)
(19.5, 7.5)
(19.5, 8.0)
(19.5, 8.5)
(19.5, 9.0)
(19.5, 9.5)


(6.5, 1.0)

In [4]:
san.fs_search(0.012, 2, 20, [Time2, Position2], 0.5, 4, 0.012)

(0.5, 0.5)
(0.5, 1.0)
(0.5, 1.5)
(0.5, 2.0)
(0.5, 2.5)
(0.5, 3.0)
(0.5, 3.5)
(0.5, 4.0)
(0.5, 4.5)
(0.5, 5.0)
(0.5, 5.5)
(0.5, 6.0)


KeyboardInterrupt: 